In [1]:
import os
from GPTModel.TrainModel import *
from UniRef50DataSet import *
from ProteinTokenizer import ProteinTokenizer, aa_to_int, end_token
from GPTModel.GPTModel import *
from GPTModel.ModelConfig import *
from GPTModel.Device import *

In [6]:
# loads a model in order to continue training a model from a checkpoint
def load_model(path, cfg):
    model = GPTModel(cfg)
    if path is not None:
        model.load_state_dict(torch.load(path, weights_only=True))
    return model

In [7]:
CHOOSE_MODEL = "gpt2-small (124M)"
cfg, _ = GetModelConfig(CHOOSE_MODEL)
#model = load_model("TrainedModels/TrainedModel.pth", cfg)
model = load_model(None, cfg) # load a new,untrained model


In [3]:
tokenizer = ProteinTokenizer() #create the tokenizer

In [5]:
# load the protein sequence data set
data_file_name = os.path.join("Datasets", f"final_dataset.dat")
with open(data_file_name, "rb") as infile:
    uniref50_data = torch.load(infile)

uniref50_data = uniref50_data - 1 #the tokens in the dataset start from 1 instead of 0

# split the data set into train and validation sets
sample_count = uniref50_data.shape[0]
train_set_ratio = 0.9
train_set_size = train_set_ratio * sample_count
train_data = uniref50_data[:int(train_set_size), :]
val_data = uniref50_data[int(train_set_size):, :]

In [8]:
# create the train and validation dataloaders
train_dataloader = create_protein_sequence_dataloader(train_data, context_length=cfg["context_length"], batch_size=10,
                         shuffle=True, drop_last=True, num_workers=0)

validation_dataloader = create_protein_sequence_dataloader(val_data, context_length=cfg["context_length"], batch_size=10,
                         shuffle=False, drop_last=False, num_workers=0)

In [9]:
# get a cuda device if the system supports it
device = GetDevice()
model.to(device)
# create the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.1)


cuda


In [14]:
#perform the training
train_losses, val_losses, track_tokens_seen = train_model_simple(model, train_dataloader, validation_dataloader, optimizer, device, 50,
                       1000, 2, tokenizer.decode(val_data[120,:1024].int().tolist()), tokenizer, "TrainedModelPretraining")

Ep 1 (Step 000000): 
Ep 1 (Step 000100): 
Ep 1 (Step 000200): 
Ep 1 (Step 000300): 
Ep 1 (Step 000400): 
Ep 1 (Step 000500): 
Ep 1 (Step 000600): 
Ep 1 (Step 000700): 
Ep 1 (Step 000800): 
Ep 1 (Step 000900): 
Ep 1 (Step 001000): 
Ep 1 (Step 001100): 
Ep 1 (Step 001200): 
Ep 1 (Step 001300): 
Ep 1 (Step 001400): 
Ep 1 (Step 001500): 
Ep 1 (Step 001600): 
Ep 1 (Step 001700): 
Ep 1 (Step 001799): Train loss 2.232, Val loss 2.116
PEKARIIEEAPEEVKVTEVVSKIGDKKTVKTTKRVIKKQKGPKQEVTEIITVEKDGEESVTTVTVSEDKSPEEVESQPIEIVELPEESTIEEIKSPDGKFKQKKITKRTIKKKVGPRVDTTQITTTHEEDKAPVISVHTIHEFTDDTFTTLDDLHEPVHAQVIEEAPEEVKVTQTVTEQGDTKTVKTTKRVIKKPKGLKQEVTEITTVENEGEEPVTTVTVTEEKSPEEGETQTTEIVELPEETFVENVESPEGKPQQKKTTKRIIKKKVGPKLETTQIITEQEDDETPVTFVHKTVELIHDTVTPLVDLLTPEKARIIEEAPEEVKVTEVVSKIGDKKTIKTTKRVIKKQKGPKQEVTEIITIEKDGEEPVTTVTVSEDKSPEEVESQPIEIVELPEESTIEEIKSPDGKPKQKKITKRTIKKKVGPRVDTTHITTTHEDDKAPVVAVHTTHEFTDDTLTTLDDLHEPVHAQVIEEAPEEVKVTQTVTEQGDTKTVKTTKRVIKKPKGLKQEVTEITTVENEGEEPVTTVTVTEEKSPEEGDTQTTEIVELPEESTIEEIKSPDGKSK